# Statistical Analysis

In this notebook, we'll use selected statistical algorithms to analyze our dataset. Specifically, we'll do the following:

* Statistical analysis
    * Distribution analysis
    * Categorical variable analysis
    * Linear Regression
    * Time-series analysis
    * Outlier detection
* Predictive analysis
    * Logistic regression
    * Random Forest
    * Support Vector Machine (SVM)
* Save the results
    * Save a predictive model for production use

First we'll get our data into our dataframe.

## Create a DataFrame of the Data

In [ ]:
# Import the Python libraries we need
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Define a variable for the accidents data file
accidents_data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/Stats19-Data1979-2004/Accidents7904.csv'

accidents = pd.read_csv(accidents_data_file,
                        sep=',',
                        header=0,
                        index_col=False,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
accidents.head()

In [ ]:
# Get a full list of the columns and data types
accidents.dtypes

## Perform a Distribution Analysis

A distribution analysis helps us understand the distribution of various attributes of our data.

In [ ]:
# Create a histogram of the weather conditions
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(accidents['Weather_Conditions'], 
        range = (accidents['Weather_Conditions'].min(),accidents['Weather_Conditions'].max()))
counts, bins, patches = ax.hist(accidents['Weather_Conditions'], facecolor='green', edgecolor='gray')
ax.set_xticks(bins)
plt.title('Weather Conditions Distribution')
plt.xlabel('Weather Condition')
plt.ylabel('Count of Weather Condition')
plt.show()

Per the Data Guide provided with the data, here are the corresponding meanings for the weather condition values.

* -1 - Data missing or out of range
* 1 - Fine no high winds
* 2 - Raining no high winds
* 3 - Snowing no high winds
* 4 - Fine + high winds
* 5 - Raining + high winds
* 6 - Snowing + high winds
* 7 - Fog or mist
* 8 - Other
* 9 - Unknown

In [ ]:
# Create a box plot of the light conditions
# The ';' at the end of the function call suppresses the usual matplotlib output
accidents.boxplot(column='Light_Conditions',
                  return_type='dict');

In [ ]:
# Create a box plot of the light conditions grouped by weather conditions
accidents.boxplot(column='Light_Conditions',
                  by = 'Weather_Conditions',
                  return_type='dict');

## Categorical Variable Analysis

A categorical variable analysis helps us understands categorical types of data. Categorical types are non-numeric. In this example, we're using day of the week. Technically it's a category as opposed to purely numeric data. The creators of the dataset have already converted the category - the name of the day of the week - to a number. If they hadn't done this, we could use Pandas to do it for us, and then perform our analysis.

In [ ]:
# Plot the distribution of casualties by day of the week
# Sunday = 1
casualty_count = accidents.groupby('Day_of_Week').Number_of_Casualties.count()
casualty_probability = accidents.groupby('Day_of_Week').Number_of_Casualties.sum()/accidents.groupby('Day_of_Week').Number_of_Casualties.count()
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Day of Week')
ax1.set_ylabel('Casualty Count')
ax1.set_title("Casualties by Day of Week")
casualty_count.plot(kind='bar')

ax2 = fig.add_subplot(122)
casualty_probability.plot(kind = 'bar')
ax2.set_xlabel('Day of Week')
ax2.set_ylabel('Probability of Casualties')
ax2.set_title("Probability of Casualties by Day of Week")

## Linear Regression

"In statistics, regression analysis is a statistical process for estimating the relationships among variables...More specifically, regression analysis helps one understand how the typical value of the dependent variable (or 'criterion variable') changes when any one of the independent variables is varied, while the other independent variables are held fixed."

Linear regression is an approach for predicting a quantitative response using a single feature (or "predictor" or "input variable").

For this recipe we are going to use the Advertising dataset from 'An Introduction to Statistical Learning
with Applications in R'.

In [ ]:
# Import the data
# Define a variable for the accidents data file
data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/ISL/Advertising.csv'

ads = pd.read_csv(data_file,
                        sep=',',
                        header=0,
                        index_col=False,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
ads.head()

In [ ]:
# How much data do we have?
ads.shape

In [ ]:
# Visualize the relationship between sales and TV in a scatterplot
ads.plot(kind='scatter',
         x='TV',
         y='Sales',
         figsize=(16, 8))

In [ ]:
# Import the Python libraries we need
from sklearn.linear_model import LinearRegression

# Create an instance of the LinearRegression model
lm = LinearRegression()

# Create X and y
features = ['TV', 'Radio', 'Newspaper']
x = ads[features]
y = ads.Sales

# Fit the data to the model
lm.fit(x, y)

# Print the intercept and coefficients
# Intercept: the expected mean value of Y when all X=0
# Coefficients: 

print(lm.intercept_)
print(lm.coef_)

In [ ]:
# Aggregate the feature names and coefficients to create a single object
fc = zip(features, lm.coef_)
list(fc)

In [ ]:
# Calculate the R-squared value: a statistical measure of how close the data are to the fitted regression line
# The closer to 100% this number is the better the model fits the data
lm.score(x, y)

In [ ]:
# Make a sales prediction for a new observation
# Given the ad spend for three channels how many thousands of widgets do we predict we will sell
# Dollars (in thousands) spent on tv, radio, and newspaper advertising
lm.predict([75.60, 132.70, 34])

## Time-Series Analysis

In [ ]:
# Create a dataframe containing the total number of casualties by date
casualty_count = accidents.groupby('Date').agg({'Number_of_Casualties': np.sum})

# Convert the index to a DateTimeIndex
casualty_count.index = pd.to_datetime(casualty_count.index)

# Sort the index so the plot looks correct
casualty_count.sort_index(inplace=True,
                          ascending=True)

# Plot the data
casualty_count.plot(figsize=(18, 4))

In [ ]:
# Plot one year of the data
casualty_count['2000'].plot(figsize=(18, 4))

In [ ]:
# Plot the yearly total casualty count for each year in the 1980's
the1980s = casualty_count['1980-01-01':'1989-12-31'].groupby(casualty_count['1980-01-01':'1989-12-31'].index.year).sum()
the1980s

In [ ]:
# Show the plot
the1980s.plot(kind='bar',
              figsize=(18, 4))

In [ ]:
# Plot the 80's data as a line graph to better see the differences in years
the1980s.plot(figsize=(18, 4))

## Outlier Detection

Outlier detection is used to find outliers in the data that can throw off your analysis.

Outliers come in two flavors: Univariate and Multivariate. Univariate outliers can be seen when looking at a single variable; multivariate outliers are found in multi-dimensional data.

For this recipe we are going to use the College dataset from 'An Introduction to Statistical Learning
with Applications in R'.

In [ ]:
# Import the dataset
data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/ISL/College.csv'

# Use the first column as the index - the dataset is set up to be like this
colleges = pd.read_csv(data_file,
                        sep=',',
                        header=0,
                        index_col=0,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
colleges.head()

In [ ]:
colleges.dtypes

In [ ]:
colleges.shape

In [ ]:
# View a boxplot of the number of applications and the number of accepted applicants
colleges.boxplot(column=['Apps', 'Accept'],
                 return_type='axes',
                 figsize=(12,6))

In [ ]:
# Visualize the relationship between the application and acceptance numbers in a scatterplot
colleges.plot(kind='scatter',
              x='Accept',
              y='Apps',
              figsize=(16, 6))

In [ ]:
# Label each point so we can see which points are the outliers
# Except for the outliers, this will be completely unreadable

# Create the plot
fig, ax = plt.subplots()

colleges.plot(kind='scatter',
              x='Accept',
              y='Apps',
              figsize=(16, 6),
              ax=ax)

# Label each of the points
for k, v in colleges.iterrows():
    ax.annotate(k,(v['Accept'],v['Apps']))

# Re-draw the scatterplot
fig.canvas.draw()

## Logistic Regression

Logistic Regression is a statistical technique used to predict a binary outcome, for example purchase/no-purchase.

For this recipe we are going to use the Heart dataset from 'An Introduction to Statistical Learning with Applications in R'.

In [ ]:
# Import the dataset
data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/ISL/Heart.csv'

# Use the first column as the index - the dataset is set up to be like this
heart = pd.read_csv(data_file,
                        sep=',',
                        header=0,
                        index_col=0,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
heart.head()

In [ ]:
heart.dtypes

In [ ]:
heart.shape

In [ ]:
# Convert the ChestPain column to a numeric value
t2 = pd.Series({'asymptomatic' : 1,
                'nonanginal' : 2,
                'nontypical' : 3,
                'typical': 4})
heart['ChestPain'] = heart['ChestPain'].map(t2)
heart.head()

In [ ]:
# Convert the Thal column to a numeric value
t = pd.Series({'fixed' : 1,
               'normal' : 2,
               'reversible' : 3})
heart['Thal'] = heart['Thal'].map(t)
heart.head()

In [ ]:
# Convert the AHD column to a numeric value
t = pd.Series({'No' : 0,
               'Yes' : 1})
heart['AHD'] = heart['AHD'].map(t)
heart.head()

In [ ]:
# Fill missing values in with 0
heart.fillna(0, inplace=True)
heart.head()

In [ ]:
# What is the shape of the data?
heart.shape

In [ ]:
# Create two matrices for our model to use
heart_data = heart.iloc[:,0:13].values
heart_targets = heart['AHD'].values

In [ ]:
# Build the model
from sklearn import linear_model
logClassifier = linear_model.LogisticRegression(C=1, random_state=111)

# Add in cross validation for our model
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(heart_data,
                                                                     heart_targets,
                                                                     test_size=0.20,
                                                                     random_state=111)
logClassifier.fit(X_train, y_train)

In [ ]:
# Estimate the accuracy of the model on our dataset
# Splits the data, fits the model and computes the score 12 consecutive times with different splits each time
scores = cross_validation.cross_val_score(logClassifier, heart_data, heart_targets, cv=12)
scores

In [ ]:
# Show the mean accuracy score and the standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# Run the test data
predicted = logClassifier.predict(X_test)
predicted

In [ ]:
# Evaluate the model
from sklearn import metrics
metrics.accuracy_score(y_test, predicted)

In [ ]:
# View the confusion matrix

# Confusion matrix - shows the predictions that the model made on the test data
# Diagonal from top-left corner to bottom-right corner is number of correct predictions for each row
# A number in a non-diagonal row is the count of errors for that row, and the column corresponds to the incorrect prediction

metrics.confusion_matrix(y_test, predicted)

## Random Forest

A random forest is an ensemble (a group) of decision trees which will output a prediction value

In [ ]:
# Import the dataset
data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/ISL/Heart.csv'

# Use the first column as the index - the dataset is set up to be like this
heart = pd.read_csv(data_file,
                        sep=',',
                        header=0,
                        index_col=0,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
heart.head()

In [ ]:
# Convert the ChestPain column to a numeric value
t2 = pd.Series({'asymptomatic' : 1,
                'nonanginal' : 2,
                'nontypical' : 3,
                'typical': 4})
heart['ChestPain'] = heart['ChestPain'].map(t2)

# Convert the Thal column to a numeric value
t = pd.Series({'fixed' : 1,
               'normal' : 2,
               'reversible' : 3})
heart['Thal'] = heart['Thal'].map(t)

# Convert the AHD column to a numeric value
t = pd.Series({'No' : 0,
               'Yes' : 1})
heart['AHD'] = heart['AHD'].map(t)

# Fill missing values in with 0
heart.fillna(0, inplace=True)
heart.head()

In [ ]:
# Import the random forest library
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
rfClassifier = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the AHD labels and create the decision trees
rfClassifier = rfClassifier.fit(X_train, y_train)
rfClassifier

In [ ]:
# Take the same decision trees and run it on the test data
predicted = rfClassifier.predict(X_test)
predicted

In [ ]:
# Estimate the accuracy of the model on our dataset
scores = cross_validation.cross_val_score(rfClassifier, heart_data, heart_targets, cv=12)
scores

In [ ]:
# Show the mean accuracy score and the standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# Assess the model
metrics.accuracy_score(y_test, predicted)

In [ ]:
# Show the confusion matrix
metrics.confusion_matrix(y_test, predicted)

## Support Vector Machine (SVM)

Support Vector Machines (SVM) are a group of supervised learning methods that can be applied to classification or regression.

For this recipe we are going to use the Heart dataset from 'An Introduction to Statistical Learning with Applications in R'.

In [ ]:
# Import the dataset
data_file = '/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Data/ISL/Heart.csv'

# Use the first column as the index - the dataset is set up to be like this
heart = pd.read_csv(data_file,
                        sep=',',
                        header=0,
                        index_col=0,
                        parse_dates=True,
                        tupleize_cols=False,
                        error_bad_lines=False,
                        warn_bad_lines=True,
                        skip_blank_lines=True,
                        low_memory=False
                        )
heart.head()

In [ ]:
# Convert the ChestPain column to a numeric value
t2 = pd.Series({'asymptomatic' : 1,
                'nonanginal' : 2,
                'nontypical' : 3,
                'typical': 4})
heart['ChestPain'] = heart['ChestPain'].map(t2)

# Convert the Thal column to a numeric value
t = pd.Series({'fixed' : 1,
               'normal' : 2,
               'reversible' : 3})
heart['Thal'] = heart['Thal'].map(t)

# Convert the AHD column to a numeric value
t = pd.Series({'No' : 0,
               'Yes' : 1})
heart['AHD'] = heart['AHD'].map(t)


# Fill missing values in with 0
heart.fillna(0, inplace=True)
heart.head()

In [ ]:
# Create an instance of a linear support vector classifier, an SVM classifier
from sklearn.svm import LinearSVC
svmClassifier = LinearSVC(random_state=111)
svmClassifier

In [ ]:
# Train the model - the svmClassifier we created earlier - with training data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(heart_data,
                                                                     heart_targets,
                                                                     test_size=0.20,
                                                                     random_state=111)
svmClassifier.fit(X_train, y_train)

In [ ]:
# Run the test data through our model by feeding it to the predict function of the model 
predicted = svmClassifier.predict(X_test)
predicted

In [ ]:
# Estimate the accuracy of the model on our dataset
scores = cross_validation.cross_val_score(rfClassifier, heart_data, heart_targets, cv=12)
# Show the mean accuracy score and the standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# Assess the model
metrics.accuracy_score(y_test, predicted)

In [ ]:
# Show the confusion matrix
metrics.confusion_matrix(y_test, predicted)

## Save the Models for Production Use

In [ ]:
# Import the Python libraries we need
import pickle

In [ ]:
# Logistic Regression Model
hearts_classifier_file = "/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Models/hearts_lr_classifier_09.27.15.dat"
pickle.dump(logClassifier, open(hearts_classifier_file, "wb"))

In [ ]:
# Random Forest Model
hearts_classifier_file = "/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Models/hearts_rf_classifier_09.27.15.dat"
pickle.dump(rfClassifier, open(hearts_classifier_file, "wb"))

In [ ]:
# SVM Model
hearts_classifier_file = "/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Models/hearts_svm_classifier_09.27.15.dat"
pickle.dump(svmClassifier, open(hearts_classifier_file, "wb"))

In [ ]:
# Reconstitute the logistic regression model as a test
model_file = "/Users/robertdempsey/Dropbox/private/Python Business Intelligence Cookbook/Models/hearts_lr_classifier_09.27.15.dat"
logClassifier2 = pickle.load(open(model_file, "rb"))
print(logClassifier2)